In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0' # Set this to a single, unused gpu
# os.environ['CUDA_VISIBLE_DEVICES']='-1' # Use -1 for CPU only, this will make things a bit slower

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#### Convenience decision boundaries plot
X are actual points in feature/embedding space (2-dim), Y are their labels
predict should return classes for an iterable of 2-dim points

In [ ]:
from matplotlib.colors import Normalize
def plot_class_regions(X, Y, nclasses, predict):
    plt.figure(figsize=(14, 14))
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = 1000.0
    xx, yy = np.meshgrid(np.arange(x_min, x_max, (x_max-x_min)/h),
                         np.arange(y_min, y_max, (y_max-y_min)/h))
    Z = predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    Z = Z / nclasses
    Y = Y / nclasses
    cmap = 'rainbow' # Whatever makes you feel pretty https://matplotlib.org/examples/color/colormaps_reference.html
    cs = plt.contourf(xx, yy, Z, np.arange(nclasses)/nclasses, cmap=cmap, antialiased=True, vmin=0, vmax=1.)
    plt.contour(cs, colors='w', antialiased=True, linewidths=0.1)
    # plt.axis('off') # If you want the looks without the detail
    # Plot the provided points
    plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='w', s=20, cmap=cmap, vmin=0, vmax=1.)
    
    handles = [create_proxy(i / nclasses, str(i), cmap) for i in range(nclasses)]
    plt.legend(handles=handles)
    plt.show()
    
def create_proxy(color, label, cmap):
    cmap = plt.get_cmap(cmap)
    import matplotlib.patches
    patch = matplotlib.patches.Patch(color=cmap(color), label=label)
    return patch

#### Base model definition
We will be working with degenerated auto-encoders so that we can use really simple data. Auto-encoder is meant to output the same data it receives, with some magic happening inbetween. The magic relates to Encoder-Decoder components that compose an auto-encoder and we will be able to train end-to-end. Encoder takes some input and outputs a vectorized representation of that input (a point in n-dimensional embedding/semantic space), while the Decoder takes that representation and tries to reconstruct the original input.

We will use auto-encoders with various twists, but the data we will be working with are simple class labels (something that does not make much sense for auto-encoders).
Do note that parts of our auto-encoder can be found in most NN models in some shape or form. Decoder which reconstructs class labels is your everyday classifier, while the Encoder can be just about any neural network with its top sliced off at some layer or a simple embedding table.

The main focus of this exercise is to see the relation between a Neural Network layer output (and its output space) and final model predictions. 

The base model bellow is just to remove some boiler-plate later on. The main attraction is the embeddings table that replaces an actual encoder and makes some things easier (embedding table serves as a perfect encoder).

In [ ]:
class BaseAutoEncoder(object):
    def __init__(self, num_classes, embedding_size=2): # Make sure inheriting models have these 2 args first in their constructor
        self.num_classes = num_classes
        self.embedding_size = 2
        self.y = tf.placeholder(dtype=tf.int64, shape=(None,)) # Class labels, sparse, None dimension means its inferred dymanically when the tensor is provided
        self.embeddings = tf.get_variable('embeddings', [num_classes, embedding_size],
                                          dtype=tf.float64, initializer=tf.random_normal_initializer)
        self.embedded_input = self.make_embedded_input()
        self.logits = self.make_logits(self.embedded_input)
        sample_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.y, logits=self.logits) # Per batch element loss
        self.loss = tf.reduce_mean(sample_loss)
        self.prob = tf.nn.softmax(self.logits) # defaults to softmax over last dimension
        self.pred = tf.argmax(self.logits, axis=1)
        self.acc = tf.reduce_sum(tf.cast(tf.equal(self.y, self.pred), dtype=tf.int32)) / tf.shape(self.y)[0] # tf.shape returns dynamic shape, inferred None's
        
        # Ops for evaluating any embedding, we need a separate graph with some weight reuse
        self.emb_input = tf.placeholder(dtype=tf.float64, shape=(None, embedding_size))
        self.emb_logits = self.make_logits(self.emb_input, reuse=True)
        self.emb_pred = tf.argmax(self.emb_logits, axis=1)
    # Implement this when inheriting
    def make_logits(self, embedded_input, reuse=False):
        raise NotImplementedError
    def make_embedded_input(self):
        raise NotImplementedError

#### Experiment wrapping func
More boilerplate, we do a bunch of stuff here.
We train our model on x, y provided by get_xy

In [ ]:
def experiment(nclasses, nsteps, model_cls, get_xy, batch_size=32, test_size=1024, **kwargs):
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    model = model_cls(nclasses, 2, **kwargs)
    train_op = tf.train.AdamOptimizer().minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    def predict(X): # A wrap for plotting
        res = []
        chunksize = 1024
        for i in range(X.shape[0] // chunksize + 1):
            res.append(sess.run(model.emb_pred, feed_dict={model.emb_input: X[i*chunksize:(i+1)*chunksize]}))
        return np.concatenate(res)
    for i in range(nsteps+1):
        x, y = get_xy(batch_size, nclasses) # Pull out a batch of data
        _, loss, acc, emb_x = sess.run([train_op, model.loss, model.acc, model.embedded_input],
                                feed_dict={
                                    model.x: x,
                                    model.y: y,
                                })
        if i % (nsteps / 5) == 0: # Five times during training we will have at whats going on
            print("Step {} acc: {}% loss: {}".format(i, acc * 100, loss))
            plot_class_regions(emb_x, y, nclasses, predict)
    test_x, test_y = get_xy(test_size, nclasses)
    test_loss, test_acc, test_embs = sess.run([model.loss, model.acc, model.embedded_input],
                                              feed_dict={
                                                  model.x: test_x,
                                                  model.y: test_y,
                                              })
    plot_class_regions(test_embs, test_y, nclasses, predict)
    print("Acc: {}%, Loss: {}".format(test_acc * 100, test_loss))
    return sess, model

#### First actual example, yay, a basic compressing auto-encoder
Lets see how many different classes can we embed into two dimensional space and how does the decision boundary look. We call this a compressing auto-encoder since it should reduce the dimensionality of input data while still able to reconstruct it from compressed representation. We do 2-dimensional so we can plot it easily.

Embeddings, or distributed representation or vectorized representation etc., are often interesing beyond just giving them to the next NN layer as inputs.
You can view almost any layer outputs (or combination of layers, or projection of layers, it gets weird really) as an embedding/representation of original input data. There are properties of such embeddings that are kinda nice. (hint: we would really like if we could cluster them meaningfully, but nice looking decision boundaries in regard to model output are nice too)

In [ ]:
class CompressingAutoEncoder(BaseAutoEncoder):
    def __init__(self, num_classes, embedding_size=2, layers=None, activation_fn=tf.nn.sigmoid):
        self.x = tf.placeholder(dtype=tf.int64, shape=(None,))
        self.layers = layers
        self.activation_fn = activation_fn
        super().__init__(num_classes, embedding_size)
    def make_logits(self, embedded_input, reuse=False):
        with tf.variable_scope(tf.get_variable_scope(), reuse=reuse): # A simple wrapper to allow variable reuse
            if self.layers:
                # Regularization?
                embedded_input = slim.stack(embedded_input, slim.fully_connected,
                                            self.layers, activation_fn=self.activation_fn)
            logits = slim.fully_connected(embedded_input, self.num_classes,
                                          activation_fn=None, scope='logits') # fully_connected defaults to ReLU, not good for logits
        return logits
    def make_embedded_input(self):
        return tf.nn.embedding_lookup(self.embeddings, self.x)

#### First experiment
Lets embed 8 classes into 2-dimensions. Our target is simple, for class X (its 2-dim embedding) predict label X.

Re-run a couple of times and you will notice that the final solution is not the same, nor truly optimal.

Re-run with more/less classes (you might need to change nsteps). In theory we can fit an infinite amount of classes in 2-dimensional space, optimization algorithms, however, are not that smart and there are limitations. (16 should work, 32 not so much anymore)

In [ ]:
def get_xy(batch_size, nclasses):
    x = y = np.random.randint(nclasses, size=batch_size)
    return x, y
sess, _ = experiment(nclasses=8, nsteps=5000, model_cls=CompressingAutoEncoder, get_xy=get_xy, batch_size=32)
sess.close()

Okay, lets try more layers now. More layers make for a smoother decision boundaries (activation_fn affects this!), too many layers makes weird decision boundaries.

layers arg is number of neurons per hidden layer, ex. [32] means one hidden layer with 32 neurons.

More layers also make optimization algorithms job easier (unless its too many layers, go figure), try a number of classes that did not really work earlier.

Try playing around with several layer combinations, and try changing the activation_fn to tf.nn.relu (or None, if you dare). more options: https://www.tensorflow.org/api_guides/python/nn

In [ ]:
sess, _ = experiment(nclasses=8, nsteps=5000, model_cls=CompressingAutoEncoder, get_xy=get_xy, batch_size=32, layers=[32], activation_fn=tf.nn.sigmoid)
sess.close()

#### Second experiment
Ok, that was easy and fun as our model only needed to really worry about nclasses different points. It did give some nice decision area around each point (unless you did some really weird archs), but that kind of stuff is not really guaranteed.

This time we will be artificially adding some noise to our 2-dim embeddings. This is just to simulate a more realistic Encoder that does not output only nclasses fixed points, but rather some cluster (remember, inputs that are similar in an abstract sense [class label in this case] should have similar embeddings in a spatial sense [euclid distance or cosine similarity]).

In [ ]:
class NoisyCompressingAutoEncoder(BaseAutoEncoder):
    def __init__(self, num_classes, embedding_size=2, layers=None, activation_fn=tf.nn.sigmoid, stdev=1):
        self.x = tf.placeholder(dtype=tf.int64, shape=(None,))
        self.layers = layers
        self.activation_fn = activation_fn
        self.stdev = stdev
        super().__init__(num_classes, embedding_size)
    def make_logits(self, embedded_input, reuse=False):
        with tf.variable_scope(tf.get_variable_scope(), reuse=reuse):
            if self.layers:
                embedded_input = slim.stack(embedded_input, slim.fully_connected,
                                            self.layers, activation_fn=self.activation_fn)
            logits = slim.fully_connected(embedded_input, self.num_classes,
                                          activation_fn=None, scope='logits')
        return logits
    def make_embedded_input(self):
        embedded_input = tf.nn.embedding_lookup(self.embeddings, self.x)
        # Additive gaussian noise, embeddings for a class are a neat cluster instead of a single point
        embedded_input = tf.add(embedded_input, tf.random_normal(shape=[tf.shape(embedded_input)[0], self.embedding_size],
                                                                 mean=0.0, stddev=self.stdev, dtype=tf.float64))
        return embedded_input

Adding some gaussian noise to embeddings makes optimizers life harder as this time around it HAS to make a decision area around the class embedding. Still, the final output should not differ too much (check the final graph to see whats going on), unless you get too weird with parameters.

A big note is that euclidian distances (as our noise is Gaussian on each dimension) do not correspond to model output probability shifts (again, unless you train on actual Gaussian data with expressive enough model where it struggles to learn that). What that means for us is... not much unless you like math and want to read up on natural gradients, stuff just kinda tends to work while the data comes from the same distribution and you don't treat model confidences as raw probability.

A big thing was mentioned in the last sentence, so lets repeat that: "stuff just kinda tends to work while the data comes from the same distribution". Have a good look at decision areas and you will see brightly colored spaces with no actual samples in them. Neural networks are ultimately of limited capacity (or expressiveness, or the ability to approximate the ideal decision function) but have to work on infinite input spaces. They do so by working well on a piece of input space (where you have data) while everywhere else is 'Here be dragons land' and just about anything can be going on. If your stuff does not work on new data, make sure data like that was seen by your model while training and don't trust output confidences to behave nicely on unseen data.

In [ ]:
sess, _ = experiment(nclasses=8, nsteps=5000, model_cls=NoisyCompressingAutoEncoder, get_xy=get_xy, batch_size=32, layers=[32,32], activation_fn=tf.nn.sigmoid, stdev=1)
sess.close()

#### Third experiment, a weird attempt at denoising
A proper denoising auto-encoder has noise at raw input, and has to remove it (the compression part is not really important anymore).

In our weird case, we will simulate input noise by making our class label a one-hot vector (ex. [3] = [0, 0, 0, 1, 0, ... , 0]), adding some Gaussian noise (small!), and L2 normalizing the vector (so it sums up to 1). In that way we make our input a bit 'uncertain' about its class. The desired output is naturally the argmax of the noisy class vector.

In this case our embeddings, aka 'fake encoder' is a bit awkward as we do a weighted average of class embeddings (it is a bad encoder basically), but the noise is not Gaussian in the embedding space (its a bit more interesting, but not much so as points will tend to be pulled toward other class centroids, and increasing the number of classes will make thing harder).

So, for fun, raise the noise levels (stdev), and at some point the whole thing will fall apart!
Also, try softmax instead of L2 normalization.

(and the usual, hidden layers, activation_fn etc.)

In [ ]:
class DenoisingCompressingAutoEncoder(BaseAutoEncoder):
    def __init__(self, num_classes, embedding_size=2, layers=None, activation_fn=tf.nn.sigmoid, stdev=0.1):
        self.x = tf.placeholder(dtype=tf.int64, shape=(None,))
        self.layers = layers
        self.activation_fn = activation_fn
        self.stdev = stdev
        super().__init__(num_classes, embedding_size)
    def make_logits(self, embedded_input, reuse=False):
        with tf.variable_scope(tf.get_variable_scope(), reuse=reuse):
            if self.layers:
                embedded_input = slim.stack(embedded_input, slim.fully_connected,
                                            self.layers, activation_fn=self.activation_fn)
            logits = slim.fully_connected(embedded_input, self.num_classes,
                                          activation_fn=None, scope='logits')
        return logits
    def make_embedded_input(self):
        self.nx = tf.nn.l2_normalize(tf.add(tf.one_hot(self.x, self.num_classes, dtype=tf.float64), 
                                            tf.random_normal(shape=[tf.shape(self.x)[0], self.num_classes],
                                                             mean=0.0, stddev=self.stdev, dtype=tf.float64)),
                                     dim=1)     
        embedded_input = tf.matmul(self.nx, self.embeddings) # Embedding is a weighted average (an NN encoder would serve better at this point)
        return embedded_input

In [ ]:
sess, _ = experiment(nclasses=16, nsteps=5000, model_cls=DenoisingCompressingAutoEncoder, get_xy=get_xy, batch_size=32,
                  layers=[64, 32], activation_fn=tf.nn.sigmoid, stdev=0.1)
sess.close()

#### Exercise!!!

embedded_input = tf.matmul(self.nx, self.embeddings) is a bad idea, imagine self.x or self.nx is a proper feature vector and use a NN to get 2-dim embeddings instead of the per-class embedding table. You can probably raise the noise levels as well for that.


In [ ]:
# Write here

#### MNIST, the datasets that is everyone's b!tch
MNIST is a grayscale handwritten digits dataset and was the first shining example of multilayered neural networks back in 1997 (by one of ML's 'big three' LeCun) (http://yann.lecun.com/exdb/mnist/).

Nowdays, and for a while by now, countless models are showing off by hitting >99% classification accuracy on MNIST. The dataset is trivial with modern computer power, but that makes is quite nice for playing around since we can re-train models quickly.

So, lets encode them digits in 2-dimensional space! (its so easy even clustering can do it http://colah.github.io/posts/2014-10-Visualizing-MNIST/)

But first lets download the data. Its simple since soo many people use MNIST (its been around the block quite a few times) for tutorials and stuff.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
class MnistCompressingEncoder(BaseAutoEncoder): # Its no longer an auto-encoder but a classifer, but renaming and good design is hard
    def __init__(self, num_classes, embedding_size=2, enc_layers=None, dec_layers=None, enc_activation_fn=tf.nn.sigmoid, dec_activation_fn=tf.nn.sigmoid):
        self.x = tf.placeholder(dtype=tf.float64, shape=(None, 784))
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.enc_activation_fn = enc_activation_fn
        self.dec_activation_fn = dec_activation_fn
        super().__init__(num_classes, embedding_size)
    def make_logits(self, embedded_input, reuse=False):
        with tf.variable_scope(tf.get_variable_scope(), reuse=reuse):
            if self.dec_layers:
                embedded_input = slim.stack(embedded_input, slim.fully_connected,
                                            self.dec_layers, activation_fn=self.dec_activation_fn, scope='decoder')
            logits = slim.fully_connected(embedded_input, self.num_classes,
                                          activation_fn=None, scope='logits')
        return logits
    def make_embedded_input(self):
        embedded_input = self.x
        if self.enc_layers:
            # What is convolution? Dunno, but sounds like a lot of parameters I don't feel like caring about.
            embedded_input = slim.stack(embedded_input, slim.fully_connected,
                                        self.enc_layers, activation_fn=self.enc_activation_fn, scope='encoder')
        embedded_input = slim.fully_connected(embedded_input, self.embedding_size,
                                              activation_fn=None, scope='embedder')
        return embedded_input

In [ ]:
def get_xy_mnist(batch_size, nclasses):
    if nclasses != 10:
        raise NotImplementedError("Mnist has only 10 classes.")
    x, y = mnist.train.next_batch(batch_size)
    return x, np.argmax(y, axis=1)
sess, _ = experiment(nclasses=10, nsteps=10000, model_cls=MnistCompressingEncoder, get_xy=get_xy_mnist, batch_size=32)
sess.close()

Oh boy, such a simple dataset, and a 784 x 2 x 10 architecture can't solve it. 10 classes is should be easy for the decoder, right? So lets try making a better encoder.

In [ ]:
sess, _ = experiment(nclasses=10, nsteps=10000, model_cls=MnistCompressingEncoder, enc_layers=[64,32,32], get_xy=get_xy_mnist, batch_size=32)
sess.close()

Hmm, ok, lets just go nuts!

In [ ]:
sess, model = experiment(nclasses=10, nsteps=10000, model_cls=MnistCompressingEncoder, enc_layers=[128,64,32], enc_activation_fn=tf.nn.relu,
                  dec_layers=[32], dec_activation_fn=tf.nn.sigmoid, get_xy=get_xy_mnist, batch_size=32, test_size=32000)
weights = sess.run(tf.trainable_variables())
sess.close()

We are limiting the model by forcing the representation to be only 2-dimensional at some point (pretty graphs yo!). If you make a straightforward classifier, accuracy will improve. The above is by modern standards a small neural network (with only 110912 parameters/operations), and it doesn't even utilize convolutional layers that are stock and standard of modern computer vision. Try making a model that uses convolution!


But now, lets see some adversarial samples!!!! (awesome)

In [ ]:
sample, _ = mnist.train.next_batch(1)
sample = sample[0]

This is how our samples look like.

In [ ]:
plt.figure(figsize=(14, 14))
plt.imshow(sample.reshape((28,28)), cmap='gray')
plt.show()

We will quickly train a model to use.

In [ ]:
sess, model = experiment(nclasses=10, nsteps=10000, model_cls=MnistCompressingEncoder, enc_layers=[128,64,32], enc_activation_fn=tf.nn.relu,
                  dec_layers=[32], dec_activation_fn=tf.nn.sigmoid, get_xy=get_xy_mnist, batch_size=32, test_size=32000)

The magic of adversarial samples is that they are minimal visual changes to an image that make the model give high confidence misclassifications. You can also make them on non-image data, but images are the best example for this.

The process is actually quite simple. Make a mask tensor that will be added to an input image, add a L2 loss for the mask, generate loss for the desired class instead of the true one, and generate/update the gradients ONLY for the new mask tensor.

Tadaa, you have gradient descent that optimizes wrong output and minimal input modification. (Keep in mind, this is a quite basic implementation, you may need to run it a few times for it to converge to a desired solution, also try changing the L2 loss factor it plays a big role here)

In [ ]:
tf_change = tf.placeholder(dtype=tf.float64, shape=(None, 784))
min_loss = tf.add_n([model.loss, 0.1*tf.nn.l2_loss(tf_change)])
ingrad = tf.gradients(min_loss, [model.x])
x, y = get_xy_mnist(batch_size=1, nclasses=10)
change = np.array([np.zeros(784, dtype=np.float64)])
desired_class = 1
print('True class {}, desired class {}'.format(y[0], desired_class))

In [ ]:
for i in range(1000):
    grads = sess.run(ingrad, feed_dict={model.x: x+change, model.y: [desired_class], tf_change: change})
    if i % 250 == 0:
        f, ax = plt.subplots(1, 3, figsize=(18, 6))
        ax[0].imshow((x+change)[0].reshape((28,28)), cmap='gray', vmin=0, vmax=1)
        ax[1].imshow(change[0].reshape((28,28)), cmap='gray')
        pred, prob, loss = sess.run([model.pred, model.prob, min_loss], feed_dict={model.x: x+change, model.y: [desired_class], tf_change: change})
        print("Prediction: {}, loss: {}, confidence:{}".format(pred[0], loss, np.max(prob)))
        ax[2].imshow(grads[0][0, :].reshape((28,28)), cmap='gray', vmin=0, vmax=1)
    change[0] = change[0] - 0.01 * grads[0][0, :]
    plt.show()

### More to come (one day)